In [1]:
import os
from dotenv import load_dotenv
load_dotenv(override=True)
# LANGCHAIN_API_KEY = os.getenv('LANGCHAIN_API_KEY')

os.environ['LANGCHAIN_TRACING_V2'] = 'true'
os.environ['LANGCHAIN_ENDPOINT'] = 'https://api.smith.langchain.com'
 

print(os.environ['LANGCHAIN_API_KEY'])

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


# Indexing

In [2]:
from langchain_openai import OpenAIEmbeddings
from langchain.vectorstores import Chroma

# Spécifiez le même répertoire de persistance
persist_directory = 'chroma_db'

# Utilisez la même fonction d'embedding que précédemment
embedding_function = OpenAIEmbeddings()

# Chargez le vectorstore à partir du disque
vectorstore = Chroma(
    persist_directory=persist_directory,
    embedding_function=embedding_function
)

retriever = vectorstore.as_retriever(search_kwargs={"k": 1})

/tmp/ipykernel_820034/3173385306.py:11: LangChainDeprecationWarning: The class `Chroma` was deprecated in LangChain 0.2.9 and will be removed in 1.0. An updated version of the class exists in the :class:`~langchain-chroma package and should be used instead. To use it run `pip install -U :class:`~langchain-chroma` and import as `from :class:`~langchain_chroma import Chroma``.
  vectorstore = Chroma(


# Retrieval 

In [3]:
docs = retriever.get_relevant_documents("Si je roule 10000 kms en an combien je vais toucher d'ik avec un véhicule de 8 chevaux fiscaux")

print(docs)

/tmp/ipykernel_820034/3402203661.py:1: LangChainDeprecationWarning: The method `BaseRetriever.get_relevant_documents` was deprecated in langchain-core 0.1.46 and will be removed in 1.0. Use :meth:`~invoke` instead.
  docs = retriever.get_relevant_documents("Si je roule 10000 kms en an combien je vais toucher d'ik avec un véhicule de 8 chevaux fiscaux")


[Document(metadata={'source': 'data/BOFiP/documents/Contenu/Commentaire/AIS/13927-PGP/2024-07-10/data.html'}, page_content="voiture neuve comportant huit places assises. Ce véhicule est utilisé par M. W à la fois pour ses besoins professionnels et personnels. Il émet 200 grammes de CO2 par kilomètre.Le barème 2022 prévoit un tarif de 18\xa0188 €.Après application de l’abattement au bénéfice des familles nombreuses, pour un niveau d’émissions de CO2 égal à 140 grammes par kilomètre (200 CO2 g/km - 60 CO2 g/km), le montant de la taxe s’élèverait à 310 €.Après application de l’abattement pour les véhicules comportant au moins huit places, pour un niveau d’émissions de CO2 égal à 120 grammes par kilomètre (200 CO2 g/km - 80 CO2 g/km), le montant de la taxe serait nul.M. W pourra donc bénéficier d'un montant de taxe nul lors de l’immatriculation du véhicule.C. Véhicules recourant à des sources d'énergies alternatives1. Véhicules recourant exclusivement à l'électricité ou à l'hydrogène150Son

# Generation

In [4]:
from langchain_openai import ChatOpenAI
from langchain.prompts import ChatPromptTemplate

# Prompt
template = """Réponds aux questions
{context}

Question: {question}
"""

prompt = ChatPromptTemplate.from_template(template)
prompt

ChatPromptTemplate(input_variables=['context', 'question'], input_types={}, partial_variables={}, messages=[HumanMessagePromptTemplate(prompt=PromptTemplate(input_variables=['context', 'question'], input_types={}, partial_variables={}, template='Réponds aux questions\n{context}\n\nQuestion: {question}\n'), additional_kwargs={})])

In [5]:
# LLM
llm = ChatOpenAI(model_name="gpt-3.5-turbo", temperature=0)

In [6]:
from langchain import hub
prompt_hub_rag = hub.pull("rlm/rag-prompt")

In [8]:
from langchain_core.output_parsers import StrOutputParser
from langchain_core.runnables import RunnablePassthrough

rag_chain = (
    {"context": retriever, "question": RunnablePassthrough()}
    | prompt
    | llm
    | StrOutputParser()
)

rag_chain.invoke("Je dispose d'un contrat madelin pour la prevoyance et la retraite, comment fonctionne la fiscalité, Est-ce que je dois le prendre sur le compte de l'entreprise ou sur mon compte personnel pour avoir la deduction fiscale ?")

'Si vous disposez d\'un contrat Madelin pour la prévoyance et la retraite, vous devez déclarer le montant de l\'épargne retraite constituée dans le cadre de ce contrat sur votre déclaration annuelle des revenus, cadre 6 "Charges et imputations diverses", cases QS, QT et QU. Cette mention permettra le calcul de la limite de déduction PERP par l\'administration fiscale et viendra en diminution de l\'espace de déduction dont vous disposez au titre de l\'épargne retraite. Il est donc important de le prendre en compte sur votre compte personnel pour bénéficier de la déduction fiscale.'